In [1]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

In [2]:
# 영화 리뷰 데이터 load

def read_review_Data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # 헤더제거
        
    return data

In [3]:
start = time.time()

In [4]:
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_Data('./ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료 : ', time.time() - start)

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료 :  0.3533172607421875


In [5]:
# 문장 단위로 명사만 추출해 학습 입력 데이터로 만듦
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료 ', time.time() - start)

2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료  122.06953477859497


In [6]:
review_data[:3]

[['8112052', '어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '1'],
 ['8132799',
  '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.',
  '1'],
 ['4655635', '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.', '1']]

In [7]:
# Word2Vec 모델 학습
print('3) Word2Vec 모델 학습 시작')
model = Word2Vec(sentences=docs, window=4, hs=1, min_count=2, sg=1)
print('3) Word2Vec 모델 학습 완료 : ', time.time()-start)

3) Word2Vec 모델 학습 시작
3) Word2Vec 모델 학습 완료 :  135.3813397884369


    <Word2Vec 주요 Parameter>
    - sentences : Word2Vec 모델 학습에 필요한 문장 데이터. Word2Vec 모델의 입력값으로 사용
    - size : 단어 인베딩 벡터의 차원(크기)
    - window : 주변 단어 윈도우의 크기
    - hs : 0(0이 아닌 경우 음수 샘플링 사용), 1(모델 학습에 softmax 사용)
    - min_count : 단어 최소 빈도 수 제한(설정된 min_count 빈도 수 이하의 단어들은 학습하지 않음)
    - sg : 0 (CBOW 모델), 1(skip-gram 모델)
      

In [8]:
# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료', time.time() - start)

4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료 135.85242867469788


In [9]:
# 학습된 말뭉치 수, 코퍼스 내 전체 단어 수
print('corpus_cont : ', model.corpus_count)
print('corpus_total_words :', model.corpus_total_words)

corpus_cont :  200000
corpus_total_words : 1076896


**Word2Vec 모델 활용**

In [10]:
# 모델 로딩
model = Word2Vec.load('nvmc.model')
print('corpus_total_words : ', model.corpus_total_words)

corpus_total_words :  1076896


In [11]:
# '사랑' 이란 단어로 생성한 단어 임베딩 벡터
print('사랑 :', model.wv['사랑'])

사랑 : [ 0.13319007  0.1749617  -0.29070753 -0.20046428 -0.09312264 -0.00678711
  0.08030351  0.22057207 -0.02963305 -0.06509611  0.02880693 -0.63407046
  0.08102984  0.14360158  0.43318054 -0.3022646  -0.02508244 -0.2587809
 -0.01917392 -0.6519287   0.05587517  0.20653412  0.47011295  0.02330615
  0.02930665 -0.15748982 -0.34471777 -0.45922366 -0.06283179  0.10589126
  0.10633246  0.08410739 -0.02250966 -0.3265067   0.34776473 -0.01390738
 -0.0029988  -0.33786842  0.07224037 -0.05361245 -0.01534871  0.03360471
 -0.10185269  0.10134813  0.07107296  0.07780237 -0.22104833  0.31664956
 -0.16117494 -0.05236945  0.2391159  -0.10734057 -0.00694746  0.3116295
 -0.14799078  0.06035299  0.22899546  0.00814054 -0.2822393   0.30995917
  0.02515095  0.22042486  0.20116138  0.04768445 -0.20913184  0.13502018
  0.3014584   0.3812833  -0.0605698   0.46673295 -0.12101533  0.2539079
  0.5566187  -0.5101334  -0.00917756 -0.1542184  -0.28633106 -0.60613275
 -0.10837318 -0.25107646 -0.06856649  0.12751374 

In [12]:
# 단어 유사도 계산

print('일요일 = 월요일\t', model.wv.similarity(w1='일요일', w2='월요일'))
print('안성기 = 배우\t', model.wv.similarity(w1='안성기', w2='배우'))
print('대기업 = 삼성\t', model.wv.similarity(w1='대기업', w2='삼성'))
print('일요일 != 삼성\t', model.wv.similarity(w1='일요일', w2='삼성'))
print('히어로 != 삼성\t', model.wv.similarity(w1='히어로', w2='삼성'))


일요일 = 월요일	 0.69238615
안성기 = 배우	 0.6675516
대기업 = 삼성	 0.5530728
일요일 != 삼성	 0.19621462
히어로 != 삼성	 0.18473688


In [13]:
# 가장 유사한 단어 추출
print(model.wv.most_similar('안성기', topn=5))
print(model.wv.most_similar('시리즈', topn=5))

[('설경구', 0.7566853761672974), ('장미희', 0.7545118927955627), ('씨야', 0.7508946061134338), ('문소리', 0.7417252659797668), ('김정학', 0.7397008538246155)]
[('엑스맨', 0.8012689352035522), ('캐리비안의 해적', 0.783246636390686), ('더 울버린', 0.7690298557281494), ('비포 선셋', 0.7629704475402832), ('X맨', 0.7595413327217102)]


     gensim 패키지의 model.wv.most_similar() 함수를 호출하여, 두 단어간 유사도 계산
     topn 인자는 반환되는 유사한 단어 수 의미 (현재 위에서는 5개 단어 반환)

###  `임베딩`  : 자연어를 컴퓨터 연산에 효율적인 벡터 형태로 변환하는 과정


    - 말뭉치의 품질과 충분한 데이터 양이 훌륭한 품질의 임베딩 모델을 구축할 수 있음
    - 이러한 임베딩은 사람이 이해하는 정보를 컴퓨터가 이해할 수 있는 형태로 변환하여, 신경망 모델의 입력으로 많이 사용됨
        